In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


c:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [5]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [6]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["VIC","NSW"]
raceDB = build_dataset('gru_inputs_new_simple_new.npy', hidden_size ,state_filter=states, margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()
# raceDB.adjust_weights({"Sandown Park":10})

['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']
(1202877, 22)
(657203, 24)
Latest date = 2023-04-11 00:00:00
num_features_per_dog=20


  0%|          | 0/27391 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/92063 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


number of races = 92063, number of unique dogs = 27392
0         (Healesville, 350.0)
1               (Dubbo, 318.0)
2        (Sandown Park, 515.0)
3        (Sandown Park, 515.0)
4        (Sandown Park, 515.0)
                 ...          
92058          (Temora, 457.0)
92059          (Temora, 330.0)
92060          (Temora, 330.0)
92061          (Temora, 457.0)
92062          (Temora, 330.0)
Length: 92063, dtype: object


In [7]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 72375, Test examples 19688


In [8]:
start_date = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").date()
end_date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()-datetime.timedelta(1)
period = start_date
batches = []
while start_date<end_date:
   
    print(f"{start_date=}")
    # period = min(start_date+datetime.timedelta(365), end_date)
    period = min(start_date+datetime.timedelta(480), end_date)
    print(f"{period=}")
    batches.append((start_date,period))
    start_date=period
    #batches.append((start_date,end_date))
# batches.append((period,end_date))
print(batches)

start_date=datetime.date(2019, 12, 1)
period=datetime.date(2021, 3, 25)
start_date=datetime.date(2021, 3, 25)
period=datetime.date(2022, 7, 18)
start_date=datetime.date(2022, 7, 18)
period=datetime.date(2022, 7, 31)
[(datetime.date(2019, 12, 1), datetime.date(2021, 3, 25)), (datetime.date(2021, 3, 25), datetime.date(2022, 7, 18)), (datetime.date(2022, 7, 18), datetime.date(2022, 7, 31))]


In [9]:
batch_races_ids = [] # list of race_ids
j = 0
current_batch = []
for i,r in enumerate(raceDB.raceIDs):   
    _,end_date = batches[j]
    race_date = raceDB.racesDict[r].race_date
    if race_date>end_date:
        print(end_date)

        batch_races_ids.append(current_batch)
        current_batch = []
        j += 1
        if j>len(batches)-1:
            break
    else:
        current_batch.append(r)
print(f"Train examples {[len(x) for x in batch_races_ids]}")

2021-03-25
2022-07-18
2022-07-31
Train examples [34348, 36935, 1009]


In [10]:
train_dogs = []
train_dog_input = []
for bi, batch in enumerate(tqdm(batch_races_ids)):
    print(bi)
    print(batch)
    batch_dogs = []
    batch_dog_input = []
    for i in raceDB.dog_ids:
        dog = raceDB.dogsDict[i]
        # train = [dog.races[x] for x in dog.races.keys() if x in batch]
        train = []
        for x in dog.races.keys():
            if x in batch:
                train.extened(x)
            else:
                break

        if train:
            batch_dogs.append(dog)
            batch_dog_input.append(train)
    train_dogs.append(batch_dogs)
    train_dog_input.append(batch_dog_input)

  0%|          | 0/3 [00:00<?, ?it/s]

0
['501007024', '500543165', '501394798', '501394799', '501394789', '501394797', '501031073', '501394795', '501394793', '501394794', '501394792', '501394790', '501394788', '501394796', '501394791', '501031072', '501031062', '501031066', '501031071', '500539220', '501031065', '501031068', '500539218', '500539217', '500539216', '500543172', '500539219', '500543173', '500543174', '500543175', '500539215', '500539145', '501007019', '501007023', '500539147', '500539146', '500543166', '500539211', '500539213', '500539212', '501007015', '500539214', '500543171', '500543168', '500543169', '500543167', '500543170', '501031064', '501007021', '501007020', '501007022', '501007018', '501031069', '501031067', '501031063', '501031070', '501007017', '501007016', '498695480', '501007014', '500541192', '501343190', '500541191', '500541193', '500541185', '500541183', '500541184', '500541186', '500541187', '500541189', '500541188', '500541190', '501031079', '501031082', '501031074', '501031081', '50103108

AttributeError: 'list' object has no attribute 'extened'

In [ ]:
batch_races = [[raceDB.racesDict[r] for r in inner] for inner in batch_races_ids]

In [ ]:
print(f"Train examples {[len(x) for x in train_dogs]}")
print(f"Train examples {[len(x) for x in train_dog_input]}")
print(f"Train examples {[len(x) for x in batch_races]}")
print(f"Train examples {[len(x) for x in batch_races_ids]}")

Train examples [725, 777, 616]
Train examples [725, 777, 616]
Train examples [1319, 1580, 1052]
Train examples [1319, 1580, 1052]


In [ ]:
raceDB.create_dogs_test_split_date()

  0%|          | 0/1628 [00:00<?, ?it/s]

In [ ]:
raceDB.attach_races_to_dog_inputv2()

  0%|          | 0/1629 [00:00<?, ?it/s]

In [ ]:
batchnorm = torch.nn.BatchNorm1d(84).to('cuda:0')

In [ ]:
def batch_norm_pack_seq(seq):
    x,idx = pad_packed_sequence(seq)
    print(x.shape)
    x = x.transpose(0,1).transpose(1,2)
    x = batchnorm(x.float())
    x = x.transpose(2,1).transpose(1,0)
    x = unpad_sequence(x, idx)
    x = pack_sequence(x, enforce_sorted=False)
    return x

In [ ]:
# test_idx = range(0,len(raceDB.test_dog_ids))
# # packed_x = [batch_norm_pack_seq(pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False)).to('cuda:0') for x in train_dog_input]
# packed_y = batch_norm_pack_seq(pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False))

In [ ]:
test_idx = range(0,len(raceDB.test_dog_ids))
packed_x = [pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_y = pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)

In [ ]:
train_dog_input_stacked = [[torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in x]] for x in train_dog_input]

In [ ]:
[sum([len(x) for x in n])/len([len(x) for x in n]) for n in train_dog_input_stacked]

[13.495172413793103, 15.414414414414415, 12.676948051948052]

In [ ]:
unpack_sequence(packed_x[0])[0][2]

tensor([452.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   1.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          4.0000,   0.0000,   1.0000,   0.0000,  25.0000,   7.0000, 340.0000,
          5.0000,  17.1198,  -1.0000,  -1.0000,   2.0000,   0.0000,   0.0000,
          2.0000,   1.0000,  24.2000,  19.8600,  -1.0000,  21.00

In [ ]:
raceDB.batches = {'num_batches':len(train_dogs),'dogs':train_dogs,'train_dog_input':train_dog_input,'batch_races':batch_races,'batch_races_ids':batch_races_ids,'packed_x':packed_x, 'packed_y':packed_y}

In [ ]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      input_size = raceDB.batches['packed_x'][0].data[0].shape[0]

      model = rnn_classes.GRUNetv2(input_size,config['hidden_size'], num_layers=config['num_layers'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file

      raceDB.to_cuda()

      # criterion = nn.SmoothL1Loss(reduction='mean', beta=5)
      criterion = nn.CrossEntropyLoss(reduction='none')
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      # optimizer = optim.Adadelta(model.parameters())
      # optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])

      model = model.to(device)

      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict_gru_v6 , model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [ ]:
len(raceDB.raceIDs)
wandb_config_static = {'races':states,'hidden_size':hidden_size,'stats':raceDB.stats_cols, 'num_layers':2,'batch_size': 750, 'dropout': 0.3, 'epochs': 10_000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0005, 'loss': 'L1', 'l1_beta':0.1,   'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':7}

In [ ]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [ ]:
raceDB.reset_hidden()

In [ ]:
for race in raceDB.racesDict.values():
    race.hidden_in = 0

In [ ]:
torch.cuda.empty_cache()

In [51]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

{'races': ['TAS'], 'hidden_size': 32, 'stats': ['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price'], 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0005, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
10000
{'races': ['TAS'], 'hidden_size': 32, 'stats': ['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price'], 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0005, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'valid

  0%|          | 0/10000 [00:00<?, ?it/s]

created path


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x0000022A30AFC040> (for post_run_cell):


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\logging\__init__.py", line 1087, in emit
    self.flush()
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\logging\__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\ipykernel\kernelapp.py", line 728, in start
    self.io_loop.start()
  File "c:\Users\Nick\.

Exception: The wandb backend process has shutdown

Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\wandb\sdk\wandb_run.py", line 170, in check_status
    status_response = self._interface.communicate_stop_status()
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\wandb\sdk\interface\interface.py", line 114, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\wandb\sdk\interface\interface_shared.py", line 394, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\wandb\sdk\interface\interface_shared.py", line 225, in _communicate
    return self._communicate_asyn

In [78]:
x = torch.rand(5,8)
x

tensor([[0.6560, 0.8574, 0.8770, 0.4362, 0.2822, 0.9473, 0.4004, 0.1002],
        [0.9749, 0.5190, 0.6682, 0.5948, 0.2987, 0.1036, 0.3801, 0.5014],
        [0.0033, 0.8093, 0.1804, 0.5764, 0.0327, 0.3968, 0.0751, 0.6272],
        [0.6979, 0.3927, 0.8095, 0.9006, 0.5391, 0.5833, 0.2797, 0.3567],
        [0.3626, 0.6506, 0.7127, 0.7528, 0.8708, 0.1113, 0.9103, 0.3863]])

In [79]:
torch.argmax(x,1)

tensor([5, 0, 1, 3, 6])

In [89]:
out = torch.zeros_like(x).scatter_(1, torch.argmax(x, dim=1).unsqueeze(1), 1.)
out

tensor([[0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.]])

In [92]:
y = torch.rand(5,8)
torch.argmax(y,1)

tensor([2, 4, 1, 7, 2])

In [94]:
outpred = torch.zeros_like(y).scatter_(1, torch.argmax(y, dim=1).unsqueeze(1), 1.)
outpred

tensor([[0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0.]])

In [96]:
outpred * out

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [95]:
outpred == out

tensor([[ True,  True, False,  True,  True, False,  True,  True],
        [False,  True,  True,  True, False,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True, False,  True,  True,  True, False],
        [ True,  True, False,  True,  True,  True, False,  True]])

In [70]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":hidden_size}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [250]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.002,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [7]
        },
        "batch_before_backwards": {
            'values': [7]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_before_backwards': {'values': [7]},
                'batch_size': {'values': [7]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [250]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 32},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.002,
                                  'min': 1e-05},
                'len_data': {'value': 16728},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 16728},
  'hidden_size': {'value': 32},
  'epochs': {'values': [250]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.002},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [7]},
  'batch_before_backwards': {'values': [7]}}}

In [71]:
sweep_id = wandb.sweep(sweep_config, project="NEW GRU-sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Create sweep with ID: dndkqpq9
Sweep URL: https://wandb.ai/nickojelly/NEW%20GRU-sweeps/sweeps/dndkqpq9


wandb: Agent Starting Run: pyrt65se with config:
wandb: 	batch_before_backwards: 7
wandb: 	batch_size: 7
wandb: 	dropout: 0.3
wandb: 	epochs: 250
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 32
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0019358761554446775
wandb: 	len_data: 16728
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 7, 'batch_size': 7, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'l1_beta': 0.1, 'learning_rate': 0.0019358761554446775, 'len_data': 16728, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
250
{'batch_before_backwards': 7, 'batch_size': 7, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'l1_beta': 0.1, 'learning_rate': 0.0019358761554446775, 'len_data': 16728, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNetv2(
  (gru): GRU(84, 32, num_layers=2)
  (batchnorm): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (fc0): Linear(in_features=32, out_features=1, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=326, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(

  0%|          | 0/250 [00:00<?, ?it/s]

created path



ROI < 30,▁▄▅▅▆█▃█▇▄▆▃▂
ROI < 30 2,▁▅▅▅▆█▄█▇▆▆▃▂
accuracy,▁▃▃▄▅▇▆███▇██
loss_1,█▄▄▄▄▄▄▄▄▃▃▄▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▁▂▂▂▁▁
loss_val,█▂▂▂▂▁▁▁▁▁▁▁▁
multibet outlay < 30,█▁▁▁▁▁▁▁▁▁▁▁▁
multibet profit,▁████████████
multibet profit < 30,▁▇▇▇▇█▇██▇▇▇▇
multibet profit < 30 sd,█▁▁▁▁▁▁▁▁▁▁▁▁
ROI < 30,-0.06727
ROI < 30 2,-0.0597


wandb: Agent Starting Run: xh8641x2 with config:
wandb: 	batch_before_backwards: 7
wandb: 	batch_size: 7
wandb: 	dropout: 0.3
wandb: 	epochs: 250
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 32
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0006773297168740726
wandb: 	len_data: 16728
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 7, 'batch_size': 7, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'l1_beta': 0.1, 'learning_rate': 0.0006773297168740726, 'len_data': 16728, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
250
{'batch_before_backwards': 7, 'batch_size': 7, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'l1_beta': 0.1, 'learning_rate': 0.0006773297168740726, 'len_data': 16728, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNetv2(
  (gru): GRU(84, 32, num_layers=2)
  (batchnorm): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (fc0): Linear(in_features=32, out_features=1, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=326, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(

  0%|          | 0/250 [00:00<?, ?it/s]

created path
